#### 1. Manual Tuning
This is the simplest but most labor-intensive method. You manually change the hyperparameters and observe the results.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

D:\Anaconda-16-FEB\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
D:\Anaconda-16-FEB\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Create a simple model
def create_model(learning_rate=0.001, dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(64, input_shape=(input_dim,), activation='relu'))  # Replace `input_dim` with your input feature dimension
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))  # Replace `num_classes` with the number of output classes
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Example of training the model
model = create_model(learning_rate=0.01, dropout_rate=0.3)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

#### 2. Grid Search with Keras Tuner
Keras Tuner is a library specifically for hyperparameter tuning with Keras. It supports grid search, random search, and Bayesian optimization.

In [3]:
pip install keras-tuner



     -------------------------------------- 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras_tuner import RandomSearch

In [10]:
input_dim=14
num_classes = 3

In [11]:
# Define a model-building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(input_dim,)))  # Replace `input_dim` with your input feature dimension
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(num_classes, activation='softmax'))  # Replace `num_classes` with the number of output classes
    
    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
# Initialize the tuner
tuner = RandomSearch(build_model, 
                     objective='val_accuracy', 
                     max_trials=5, 
                     executions_per_trial=3, 
                     directory='my_dir', 
                     project_name='my_project')


In [ ]:
# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

In [ ]:
# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

#### Random search